# LAB | Intro to Machine Learning

**Load the data**

In this challenge, we will be working with Spaceship Titanic data. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [38]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [3]:
spaceship.shape

(8693, 14)

In [4]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


**Check for data types**

In [5]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [6]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
def data_exploration(df):

    # check number of rows and columns
    shape = df.shape
    print("Number of rows:", shape[0])
    print("Number of columns:", shape[1])

    # check duplicates
    check_duplicates = df.duplicated().sum()
    print("Number of duplicates:", check_duplicates)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
                        'Column': df.columns,
                        'Data Type': df.dtypes,
                        'Non-Null Count': df.notnull().sum(),
                        'Missing Values': df.isnull().sum(),
                        'Unique Values': df.nunique()
                })

    # Reset index to make 'Column' a regular column
    summary_df.reset_index(drop=True, inplace=True)

    # Display the summary DataFrame
    summary_df
    
    # check numerical columns
    numerical_columns = df.select_dtypes("number").columns
    print("\nNumerical Columns:", numerical_columns)

    # check categorical columns
    categorical_columns = df.select_dtypes("object").columns
    print("\nCategorical Columns:", categorical_columns)

    return summary_df

In [8]:
data_exploration(spaceship)

Number of rows: 8693
Number of columns: 14
Number of duplicates: 0

Numerical Columns: Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')

Categorical Columns: Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP',
       'Name'],
      dtype='object')


,Column,Data Type,Non-Null Count,Missing Values,Unique Values
0,PassengerId,object,8693,0,8693
1,HomePlanet,object,8492,201,3
2,CryoSleep,object,8476,217,2
3,Cabin,object,8494,199,6560
4,Destination,object,8511,182,3
5,Age,float64,8514,179,80
6,VIP,object,8490,203,2
7,RoomService,float64,8512,181,1273
8,FoodCourt,float64,8510,183,1507
9,ShoppingMall,float64,8485,208,1115


There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [11]:
# dropping rows with null values
spaceship = spaceship.dropna()          

In [12]:
# recheck null values

spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

**KNN**

K Nearest Neighbors is a distance based algorithm, and requeries all **input data to be numerical.**

Let's only select numerical columns as our features.

In [14]:
features = spaceship.select_dtypes("number")
features.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0.0,0.0,0.0,0.0,0.0
1,24.0,109.0,9.0,25.0,549.0,44.0
2,58.0,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,303.0,70.0,151.0,565.0,2.0


## Classification

And also lets define our target.

In [15]:
target = spaceship['Transported']
target.head()

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

**Train Test Split**

Now that we have split the data into **features** and **target** variables and imported the **train_test_split** function, split X and y into X_train, X_test, y_train, and y_test. 80% of the data should be in the training set and 20% in the test set.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0) 

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

You need to choose between **Classificator** or **Regressor**. Take into consideration target variable to decide.

Initialize a KNN instance without setting any hyperparameter.

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)

Fit the model to your data.

In [19]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [20]:
# create predictions for new data points
pred = knn.predict(X_test)

# to compare the true labels
y_test.values

array([ True, False,  True, ...,  True,  True,  True])

Evaluate your model.

In [21]:
knn.score(X_test, y_test)

0.735249621785174

Precision method

In [23]:
precision = precision_score(y_test, pred, average='macro')
print(f'Precision: {precision}')

Precision: 0.7354242012656981


Recall method

In [24]:
recall = recall_score(y_test, pred, average='macro')
print(f'Recall: {recall}')

Recall: 0.735249621785174


F1 score

In [25]:
f1 = f1_score(y_test, pred, average='macro')
print(f'F1-score: {f1}')

F1-score: 0.7352005310868968


## Regression

In [26]:
# create a new column 'Total_spending'

spaceship['total_spending'] = spaceship[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)

In [27]:
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,total_spending
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,736.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,10383.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,5176.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1091.0


In [29]:
target = spaceship['total_spending']
target.head()

0        0.0
1      736.0
2    10383.0
3     5176.0
4     1091.0
Name: total_spending, dtype: float64

**Train Test Split**

In [30]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0) 

**Model Selection**

In [31]:
knn = KNeighborsRegressor(n_neighbors=10)

In [32]:
knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

Evaluate your model.

In [33]:
y_predict= knn.predict(X_test)

In [34]:
knn.score(X_test, y_test)

0.9500118966617889

In [35]:
mse_standardized = mean_squared_error(y_test, y_predict)
mae_standardized = mean_absolute_error(y_test, y_predict)
r2_standardized = r2_score(y_test, y_predict)

mse_standardized, mae_standardized, r2_standardized

(375019.35679273825, 138.00620272314674, 0.9500118966617889)

#### Standardize the data after split the data

In [39]:
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train) # fit and transform at the same time ONLY WITH TRAIN DATA
X_test_standardized = scaler.transform(X_test)

In [40]:
normalizer = MinMaxScaler()
X_train_normalized = normalizer.fit_transform(X_train)
X_test_normalized = normalizer.transform(X_test)

Train and Evaluate KNN Regressor on standardized data.

In [41]:
knn_standardized = KNeighborsRegressor(n_neighbors=2)
knn_standardized.fit(X_train_standardized, y_train)

KNeighborsRegressor(n_neighbors=2)

In [42]:
y_pred_standardized = knn_standardized.predict(X_test_standardized)

In [43]:
mse_standardized = mean_squared_error(y_test, y_pred_standardized)
mae_standardized = mean_absolute_error(y_test, y_pred_standardized)
r2_standardized = r2_score(y_test, y_pred_standardized)

mse_standardized, mae_standardized, r2_standardized

(256189.15885022693, 149.33282904689864, 0.9658513356316905)

Train and Evaluate KNN Regressor on normalized data.

In [44]:
knn_normalized = KNeighborsRegressor(n_neighbors=2)
knn_normalized.fit(X_train_normalized, y_train)
y_pred_normalized = knn_normalized.predict(X_test_normalized)
mse_normalized = mean_squared_error(y_test, y_pred_normalized)
mae_normalized = mean_absolute_error(y_test, y_pred_normalized)
r2_normalized = r2_score(y_test, y_pred_normalized)

Train and Evaluate KNN Regressor on Non-Transformed Data

In [45]:
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
mse_non_transformed = mean_squared_error(y_test, y_pred)
mae_non_transformed = mean_absolute_error(y_test, y_pred)
r2_non_transformed = r2_score(y_test, y_pred)

Display Results

In [46]:
results = {
    'Data Transformation': ['Non-Transformed', 'Standardized', 'Normalized'],
    'MSE': [mse_non_transformed, mse_standardized, mse_normalized],
    'MAE': [mae_non_transformed, mae_standardized, mae_normalized],
    'R^2': [r2_non_transformed, r2_standardized, r2_normalized]
}
results_df = pd.DataFrame(results)

# Calculate the improvement percentages compared to the non-transformed data
mse_non_transformed = results_df.loc[results_df['Data Transformation'] == 'Non-Transformed', 'MSE'].values[0]
mae_non_transformed = results_df.loc[results_df['Data Transformation'] == 'Non-Transformed', 'MAE'].values[0]
r2_non_transformed = results_df.loc[results_df['Data Transformation'] == 'Non-Transformed', 'R^2'].values[0]

# Improvements for Standardized data
mse_standardized_improvement = (mse_non_transformed - results_df.loc[results_df['Data Transformation'] == 'Standardized', 'MSE'].values[0]) / mse_non_transformed * 100
mae_standardized_improvement = (mae_non_transformed - results_df.loc[results_df['Data Transformation'] == 'Standardized', 'MAE'].values[0]) / mae_non_transformed * 100
r2_standardized_improvement = (results_df.loc[results_df['Data Transformation'] == 'Standardized', 'R^2'].values[0] - r2_non_transformed) / r2_non_transformed * 100

# Improvements for Normalized data
mse_normalized_improvement = (mse_non_transformed - results_df.loc[results_df['Data Transformation'] == 'Normalized', 'MSE'].values[0]) / mse_non_transformed * 100
mae_normalized_improvement = (mae_non_transformed - results_df.loc[results_df['Data Transformation'] == 'Normalized', 'MAE'].values[0]) / mae_non_transformed * 100
r2_normalized_improvement = (results_df.loc[results_df['Data Transformation'] == 'Normalized', 'R^2'].values[0] - r2_non_transformed) / r2_non_transformed * 100

# Display improvement percentages
improvements = {
    'Data Transformation': ['Standardized', 'Normalized'],
    'MSE Improvement (%)': [mse_standardized_improvement, mse_normalized_improvement],
    'MAE Improvement (%)': [mae_standardized_improvement, mae_normalized_improvement],
    'R^2 Improvement (%)': [r2_standardized_improvement, r2_normalized_improvement]
}
improvements_df = pd.DataFrame(improvements)

In [48]:
pd.DataFrame(results)

,Data Transformation,MSE,MAE,R^2
0,Non-Transformed,168652.313162,94.182300,0.977520
1,Standardized,256189.158850,149.332829,0.965851
2,Normalized,403706.392965,201.944781,0.946188


**Congratulations, you have just developed your first Machine Learning model!**